#### Import Parent Module

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..', '..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from jupyter_notebook import load_parameters

In [ ]:
import pandas as pd
import os
import ast
import pyflakes

#### Read Luigi Parameters

In [ ]:
pars = load_parameters()

input_path = pars.get('input')
output_file = pars.get('output')

input_col = pars.get('input_col')
output_col = pars.get('output_col')

In [ ]:
dataset = pd.read_csv(input_path, encoding='ISO-8859-1', error_bad_lines=False, sep=";")

#### Normalization Method: Add Pandas Import

**Simple heuristic:** 

1. If ```pd``` module is used but not imported -> we add the import in the beginning of the code.
2. If ```DataFrame``` type is used but not imported -> add the import of the type 


In [1]:
def normalizePandasImport(post, col):
    strcodes = post[col]
    codes = ast.literal_eval(strcodes)
    processed = []
    normalized = False
    for code in codes:
        if "pd." in code and "import pandas as pd" not in code:
            code = "import pandas as pd\n" + code
            normalized = True
            
        if " DataFrame" in code and "import DataFrame" not in code:
            code = "from pandas import DataFrame\n" + code
            normalized = True
            
        processed.append(code)
        
    post[output_col] = processed
    post['PandasNormalized'] = normalized
    
    return post

In [ ]:
normalized_dataset = dataset.apply(normalizePandasImport, axis=1, args=(input_col,))

#### Save to disk

In [ ]:
normalized_dataset.to_csv(output_file, encoding='ISO-8859-1', sep=';', doublequote=True, index=False)